In [ ]:
import re
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
DATASET_NAME = 'libraries'
MAX_TOKENS = 200
_DATA_DIR = f'../data/{DATASET_NAME}'

## 1. Load data

In [ ]:
df = pd.read_json(_DATA_DIR + '/libraries.json')
df.head()

In [ ]:
# Optional - filter out rows where the method name doesnt contain any of the chosen subtokens
# classes = { 0: 'train', 1: 'save', 2: 'process', 3: 'forward', 4: 'predict' }

# df = df[df.method_name.str.contains("|".join(classes.values()))]

# Assign categories based on method name
# df['category'] = df.method_name.map(lambda x: np.array([x.find(s) for s in classes.values()]).argmax())
# df

In [ ]:
def get_project_name(path):
    s = path.split('/')
    return f"{s[5]}--{s[6]}"

In [ ]:
# Get unique libraries
df['libraries'] = df.references.map(lambda x: list(set([s.split('.')[0] for s in x])))
# df['references'] = df['references'].apply(lambda x: list(set(x))) # Dont store duplicates
df['references'] = df['references'].apply(lambda x: x if len(x) <= MAX_TOKENS else random.sample(x, MAX_TOKENS))
df['project'] = df.file.map(lambda x: get_project_name(x))
df.head()

## 2. Briefly inspect the data

In [ ]:
stats = pd.DataFrame(np.concatenate(df.libraries.values).ravel(), columns=['library'])
stats.head()
stats['count'] = stats.groupby('library')['library'].transform('count')
stats = stats.drop_duplicates(subset=['library']).sort_values(by=['count'], ascending=False).reset_index(drop=True)
stats

In [ ]:
imports = np.concatenate(df.references.values).ravel()
print(f"Total number of methods {len(df)}\nNumber of unique method names {len(np.unique(df.method_name))}")
print(f"Total number of libraries {len(stats)}\nTotal number of import references {len(imports)}\nTotal number of unique import references {len(np.unique(imports))}")

## 3. Preprocess

In [ ]:
# Aggregate libraries by project name
project_libraries = df.groupby(['project'])['libraries'].sum().map(lambda x: list(set(x)))
project_libraries

In [ ]:
# Check frequency of libraries
df_proj_libs = pd.DataFrame(np.concatenate(project_libraries).ravel(), columns=['library'])
df_proj_libs['count'] = df_proj_libs.groupby('library')['library'].transform('count')
df_proj_libs = df_proj_libs.drop_duplicates(subset=['library']).sort_values(by=['count'], ascending=False).reset_index(drop=True)
df_proj_libs

In [ ]:
# Get all libraries when are referenced > 1 (i.e. not project specific)
shared_project_libraries = df_proj_libs[df_proj_libs['count'] > 1]
print(f"{len(shared_project_libraries)} out of {len(stats)} declared libraries are project specific ({len(shared_project_libraries) / len(stats):.2f}%)")
shared_libs = list(shared_project_libraries.library)
shared_project_libraries

In [ ]:

def camel_case_split(identifier, joinToken):
    matches = re.finditer(
        '.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)',
        identifier,
    )
    return f'{joinToken}'.join([m.group(0).lower() for m in matches])

def snake_case_split(identifier, joinToken):
    return f'{joinToken}'.join([x for x in identifier.split('_') if x != ''])

In [ ]:
df_processed = df.copy()
# Drop all library references that are project specific
df_processed['references'] = df_processed['references'].apply(lambda x: [s for s in x if s.split('.')[0] in shared_libs])
df_processed['libraries'] = df_processed['libraries'].apply(lambda x: [s for s in x if s in shared_libs])
# Some rows may now include no references - drop these
df_processed = df_processed[~df_processed.references.str.len().eq(0)]
df_processed['references'] = df_processed['references'].apply(lambda x: ",".join(x))

# split camel/snake case method names
df_processed['method_name'] = df_processed.method_name.map(lambda x: snake_case_split(x, '|'))
df_processed['method_name'] = df_processed.method_name.map(lambda x: camel_case_split(x, '|'))

df_processed

In [ ]:
tokens = df_processed['references'].str.split(',').values

num_tokens_per_method = [len(l) for l in tokens]
print(f"Max number of tokens in method {np.max(num_tokens_per_method)}\nMin number of tokens in a method {np.min(num_tokens_per_method)}\nAverage number of tokens per method {np.mean(num_tokens_per_method):.2f}")

In [ ]:
print(f"Number of unique method names: {len(np.unique(df_processed.method_name))}\nNumber of unique tokens {len(np.unique(tokens))}")

In [ ]:
df_processed = df_processed.method_name.to_frame().merge(df_processed.references, left_index=True, right_index=True)

## 4. Partition into sets

In [ ]:
train_size, val_size, test_size = 0.9, 0.05, 0.05
train, remainder = train_test_split(df_processed, test_size=(1-train_size), shuffle=True)
validate, test =  train_test_split(remainder, test_size=test_size/(test_size + val_size))

print(f"{len(train)} train samples\n{len(validate)} validation samples\n{len(test)} test samples")
train

In [ ]:
print(f"Number of unique method names: {len(np.unique(train.method_name))}\nNumber of unique tokens {len(np.unique(train['references'].str.split(',').values))}")

## 5. Save

In [ ]:
import csv
train.to_csv(_DATA_DIR+'/train.csv', encoding='utf-8', sep=" ", index=False, header=None, quoting = csv.QUOTE_NONE, escapechar = ' ')
validate.to_csv(_DATA_DIR+'/val.csv', encoding='utf-8', sep=" ", index=False, header=None, quoting = csv.QUOTE_NONE, escapechar = ' ')
test.to_csv(_DATA_DIR+'/test.csv', encoding='utf-8', sep=" ", index=False, header=None, quoting = csv.QUOTE_NONE, escapechar = ' ')